## SARvey - survey with SAR 

---
<img src="https://seafile.projekt.uni-hannover.de/f/69eac9bb5f86400fa1ad/?dl=1" width="512">


**SARvey** is an open-source software developed for the analysis of Interferometric Synthetic Aperture Radar (InSAR) displacement time series, particularly tailored for engineering applications. It offers a comprehensive workflow to process and analyze Synthetic Aperture Radar (SAR) data, enabling users to monitor and assess ground deformations with high precision.

For more information, visit the following:
* **[SARvey GitHub Repository](https://github.com/luhipi/sarvey)**
* **[SARvey Documentation](https://luhipi.github.io/sarvey/docs/)**
* **[How to cite SARvey](https://sarvey.readthedocs.io/main/readme.html#how-to-cite/)**

---

This tutorial covers the SARvey processing steps for a sample dataset, as a practical example to demonstrate its main capabilities.

---

*This tutorial is prepared by **[Mahmud Haghighi](https://www.ipi.uni-hannover.de/en/haghighi/)** from the Institute of Photogrammetry and GeoInformation, Leibniz University Hannover.*



### SARvey workflow

**TODO: add short description**
<img src="https://seafile.projekt.uni-hannover.de/f/006f702937cd4e618bcb/?dl=1" width="512">




**TODO: add short description**
<img src="https://seafile.projekt.uni-hannover.de/f/39209355cabc4607bf0a/?dl=1" width="512">


### Before we start

This tutorial uses Jupyter Notebook syntax. Here are a few helpful things to know before running the cells:

- `!` at the beginning of a line is used to run shell (terminal) commands directly from the notebook. For example:
    - `!ls` — lists files in the current directory
    - `!pwd` — prints the current working directory
    - `!cd directory_name` — changes the directory (note: cd won't persist across cells unless handled carefully)
    - `!mkdir new_folder` — creates a new directory
    - `!pip install sarvey` — installs the sarvey package
    - `!sarvey -h` — displays the help message for the sarvey command-line interface.

**Note**: You do not need the `!` if you are running these commands directly in a terminal (outside of Notebook).



## Installation

SARvey is a cross-platform python-based software and can be installed on Linux, MacOS and Windows. If you are installing SARvey on your local machine, you can refer to the **[SARvey Installation](https://sarvey.readthedocs.io/main/installation.html#installation)**.

SARvey depends on other open-source software and python packages. Most of dependencies, including *MintPy* are integrated into SARvey installation and will be installed with a simple `pip` command. pip is the package installer for Python. Before installing **SARvey**, we need to install *MiaplPy* separately.

Run the following cell to install **MiaplPy**. This cell will use `pip` for the installation. Note the exclamation mark (!) at the beginning of the command.

In [ ]:
!pip install git+https://github.com/insarlab/MiaplPy.git --quiet

Run the following cell to install **SARvey** along with its required dependencies. These include **MintPy** and several other open-source packages that SARvey relies on to function properly.

In [ ]:
!pip install git+https://github.com/luhipi/sarvey --quiet

After installation, run the following cells to verify that **MiaplPy**, **MintPy** and **SARvey** are installed correctly.

If the installation is successful, you will see the usage prompt for each software.
```
usage: miaplpyApp [-h] [--dir WORKDIR] [-g] [-H] [-v] [--walltime WALL_TIME]


```

If you encounter errors like the one below, it indicates that the software is not installed properly:

```
ERROR: unknown command "miaplpyApp"
```

In [ ]:
!miaplpyApp

In [ ]:
!mintpy -h

In [ ]:
!sarvey -h

## Imports

Install additional packages required for this tutorial.

In [ ]:
# install tree
!apt-get -qq install tree

Imports essential Python libraries required for this tutorial.

In [ ]:
import os
from IPython.display import display, Image, JSON, Markdown
from matplotlib import pyplot as plt
import numpy as np
import h5py as h5
import json5 as json

## Download data

**Demo Dataset**: Masjed Soleyman Dam

This tutorial focuses on measuring the post-construction settlement of the Masjed Soleyman Dam. It is a rock-fill dam on the Karun river, opened in 2002. [Previous investigations](https://www.sciencedirect.com/science/article/pii/S0141029617311525) using GNSS and high-resolution TerraSAR-X data, have shown substantial post-construction settlement of the dam. TerraSAR-X results show that the dam undergoes a maximum deformation rate of 13 cm/year in the radar line-of-sight. In this tutorial, we will use Sentinel-1 data to retrieve the deformation of the dam.

**Dataset Highlights:**
- **Location:** Masjed Soleyman Dam, Iran
- **Data Type:** Coregistered stack of SLCs with corresponding geometry information

Specify the working directory. 
On **Google Colab**, the working directory should be
```
work_dir = '/content'
```

In [ ]:
# specify working directory. On Google Colab it should be '/content'
work_dir = '/content'

Run the following cell to downloat the data and unzip it.

In [ ]:
# Change the directory
os.chdir(work_dir)

# Download data
!wget  -N https://seafile.cloud.uni-hannover.de/f/6bf916a2fba6404ab439/?dl=1  -O Masjed_Soleyman_Dam_S1_dsc_2015_2022.zip

# Unzip data into masjed_s1 directory
!unzip -q -o Masjed_Soleyman_Dam_S1_dsc_2015_2022.zip

# Rename the extracted folder
!mv Masjed_Soleyman_Dam_S1_dsc_2015_2022 masjed_s1

# Define the project directory path as a variable
project_dir=os.path.join(work_dir, 'masjed_s1')


## Check the input data

A typical directory structure to run SARvey looks like the following:
```
📂 project_folder/
  │── 📁 inputs/               # SARvey inputs
  │     ├── slcStack.h5        # stack of resampled SLCs
  │     └── geometryRadar.h5   # geometry in Radar coordinates
  ├── 📁 outputs/              # SARvey outputs
  └── 📄 config.json           # SARvey processing configuration
```

The input to SARvey are the `slcStack.h5` and `geometryRadar.h5`.


Let's check the directory structure of the downloaded data using `tree` command.

In [ ]:
# Navigate to the project directory
os.chdir(project_dir)

# Display the directory structure in a tree-like format
!tree


***
It is a good practice to use MintPy `info.py` functions to verify the metadata of the input files before starting the SARvey processing. 

In [ ]:
## Navigate to the input directory
os.chdir(project_dir+'/inputs')

# Display metadata and structure of the 'geometryRadar.h5' file
!info.py geometryRadar.h5


In [ ]:
## Navigate to the input directory
os.chdir(project_dir+'/inputs')

# Display metadata and structure of the 'slcStack.h5' file
!info.py slcStack.h5


There are more information that you can get using `info.py`. For example:

In [ ]:
# show a list of SLC dates with numbers
!info.py slcStack.h5 --num


***
You can also use MintPy `view.py` functions to visualize the input fles before starting the SARvey processing. 

In [ ]:
## Navigate to the input directory
os.chdir(project_dir+'/inputs')

# Use the view.py command to create geometryRadar.png
!view.py geometryRadar.h5 --save --noverbose --update

# Show the image in notebook
Image(filename='geometryRadar.png')

Instead of using `view.py` you can use the view from mintpy.cli if you are an experienced user

```
# Import view
from mintpy.cli import view

# Create geometryRadar.png
view.main('geometryRadar.h5 --save --noverbose'.split())

# Show the image in notebook
Image(filename='geometryRadar.png')
```

`view.py` can also be used to show the amplitudes from the slcStack, but it might be time-consuming, depending on the data size

***
Both input files are in HDF5 format and can be examined using a simple script with the `h5py` library. First, we define `showMeanAmplitude()` function to extract mean amplitude of first 25 images in slcStack.h5 and then call the function.

In [ ]:
def showMeanAmplitude(slc_file='inputs/slcStack.h5', n=25):
    # Open the HDF5 file containing the SLC stack
    with h5.File(slc_file, 'r') as f:
        if n>=f['slc'].shape[0]:
            print(f'Image index {n} exceeds the number of available images.')
            return
        # Extract the image 0 to n from the stack
        slc_stack = f['slc'][0:n,:,:]

    # Compute mean amplitude of the complex SLC image
    slc_amp = np.mean(np.abs(slc_stack), axis=0)
    
    plt.figure(figsize=(20,3))
    plt.imshow(np.log10(slc_amp), cmap='gray')
    plt.xlabel('[Range]')
    plt.ylabel('[Azimuth]')
    plt.title(f'Mean Amplitude of image 0-{n} in dB')
    plt.colorbar()
    plt.show()

showMeanAmplitude()

## Start SARvey processing

We first run **`sarvey -h`** to see the quick help. Please also refer to the full documentation to see the details of processing.

In [ ]:
! sarvey -h

The first step in the processing workflow is to generate the configuration file using the command:

**```sarvey -f config.json 0 0 -g```**

The **`-g`** flag instructs SARvey to generate a default config.json file. Although the command includes **`0 0`** as step indices, no processing steps will actually be executed when **`-g`** is used. Instead, the command will only create the configuration file.


In [ ]:
os.chdir(project_dir)

! sarvey -f config.json 0 0 -g

List the directory contents to confirm that **config.json** was successfully created.

In [ ]:
! tree

We can open the **config.json** file in a text editor to adjust the processing parameters before starting the SARvey workflow. Alternatively, we can read and modify the file programmatically using Python.


***
we define `loadJsonConfig()` and `dumpToJsonConfig()` functions to read and modify the config file.

In [ ]:
def loadJsonConfig(config_file):
    # Load the contents of config_file into a Python dictionary
    with open('config.json', 'r') as f:
        config_dict = json.load(f)
    return config_dict

def dumpToJsonConfig(config_file, config_dict):
    # Write the config_dict to config_file
    with open(config_file, 'w') as f:
        json.dump(config_dict, f, indent=4)
        

In [ ]:
os.chdir(project_dir)

# Load the configs
config = loadJsonConfig('config.json')

# Display the configs
config


The config file has several sections.

- **`general`**

  This section includes top-level parameters such as the number of cores and the paths to the input and output data.

- **`phase_linking`**

  This section specifies the Phase Linking parameters. By default, `use_phase_linking_results: false`, i.e, no phase linking is performed.

- **`preparation`**

  This section includes network parameters and type, and window size used to estimate the temporal coherence.

- **`consistency_check`**

  This section contains parameters related to the first order points.

- **`unwrapping`**

  This section will specify parameters related to the unwrapping process.

- **`filtering`**

  This section defines the parameters for atmospheric estimation and filtering.

- **`densification`**

  This section includes the settings for second order points.


Please refer to the [documentation](https://sarvey.readthedocs.io/main/processing.html#configuration-file) for the details of each parameter.

**SARvey** supports two general modes of processing, depending on the characteristics of the displacement (spatial extent, magnitude, temporal behaviour, etc.):
- two-step unwrapping workflow
- one-step unwrapping workflow


We will modify the following parameters before starting the processing:

- general
    - output_path: 'output_default'
    - num_cores: 10

You can modify the parameters using a text editor, or run the following cell to modify the `config.json` file.

In [ ]:
os.chdir(project_dir)

# Load the configs
config = loadJsonConfig('config.json')

# Modify parameter
config['general']['output_path'] = 'output_default'
config['general']['num_cores'] = 10

dumpToJsonConfig('config.json', config)

Now verify the modified config file **`json.config`**

In [ ]:
os.chdir(project_dir)

# Load the configs
config = loadJsonConfig('config.json')
config


Run [**Step 0: Preparation**](https://sarvey.readthedocs.io/main/processing.html#step-0-preparation) including the following:

- Loading the data from the inputs/slcStack.h5

- Designing the interferogram network based on the temporal and perpendicular baselines.

    - small baseline network (`sb`) (Berardino et al. 2002),
    - small temporal baseline network (`stb`) (only consecutive images are used to form interferograms)
    - small temporal baselines + yearly interferograms (`stb_yearly`)
    - delaunay network (`delaunay`)
    - star network (`star`, single-reference network) (Ferretti et al. 2001)

- Generating a stack of interferograms

- Estimating the temporal coherence


In [ ]:
os.chdir(project_dir)

! sarvey -f config.json 0 0 